# Backtesting 
- TA-Lib or btalib for extra indicators
- Pandas for preprocessing
- optunity or hyperopt for strategy optimization
- Joblib or Ray for parallel testing


To-Do's
-# Create manager
-manager = StrategyManager()

-# Test single strategy
-manager.run_single_strategy('trend_following', 'AAPL')

-# Compare strategies
-manager.run_strategy_comparison(['momentum', 'mean_reversion'], 'MSFT')

-# Test across multiple assets
-manager.run_multi_asset_test('conservative', ['AAPL', 'GOOGL', 'TSLA'])

-# Add your custom strategy
-manager.add_custom_strategy('my_strategy', MyCustomStrategy)

In [15]:
from backtesting import Backtest, Strategy
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_ta as ta
from datetime import datetime
from abc import ABC, abstractmethod
import warnings
warnings.filterwarnings('ignore', category=FutureWarning, module='backtesting')
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)        # Don't wrap columns
pd.set_option('display.max_colwidth', None) # Show full content of each column
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
#Old 
import backtrader as bt
import yfinance as yf
import pandas as pd
from datetime import datetime

class MovingAverageCrossover(bt.Strategy):
    """
    Simple Moving Average Crossover Strategy
    - Buy when short MA crosses above long MA
    - Sell when short MA crosses below long MA
    """
    
    # Define strategy parameters
    params = (
        ('short_period', 10),    # Short moving average period
        ('long_period', 30),     # Long moving average period
        ('printlog', True),      # Print trade logs
        ('stake', 100),          # Number of shares to trade
    )
    
    def __init__(self):
        """Initialize strategy indicators and variables"""
        
        # Reference to the "close" line in the data dataseries
        self.dataclose = self.datas[0].close
        
        # Keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None
        
        # Create moving average indicators
        self.short_ma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.short_period
        )
        self.long_ma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.long_period
        )
        
        # Create crossover signal
        self.crossover = bt.indicators.CrossOver(self.short_ma, self.long_ma)
        
        # Track trade count
        self.trade_count = 0
    
    def log(self, txt, dt=None, doprint=False):
        """Logging function for strategy"""
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print(f'{dt.isoformat()}: {txt}')
    
    def notify_order(self, order):
        """Receive order notifications"""
        if order.status in [order.Submitted, order.Accepted]:
            # Order submitted/accepted - no action required
            return
        
        # Check if order is completed
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED - Price: {order.executed.price:.2f}, '
                        f'Cost: {order.executed.value:.2f}, '
                        f'Commission: {order.executed.comm:.2f}')
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:
                self.log(f'SELL EXECUTED - Price: {order.executed.price:.2f}, '
                        f'Cost: {order.executed.value:.2f}, '
                        f'Commission: {order.executed.comm:.2f}')
            
            self.trade_count += 1
            
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log(f'Order Canceled/Margin/Rejected')
        
        # Reset order
        self.order = None
    
    def notify_trade(self, trade):
        """Receive trade notifications"""
        if not trade.isclosed:
            return
        
        self.log(f'OPERATION PROFIT - Gross: {trade.pnl:.2f}, Net: {trade.pnlcomm:.2f}')
    
    def next(self):
        """Main strategy logic - called for each bar"""
        
        # Log current close price
        self.log(f'Close: {self.dataclose[0]:.2f}')
        
        # Check if we have a pending order
        if self.order:
            return
        
        # Check if we are in the market
        if not self.position:
            # Not in market - look for buy signal
            if self.crossover[0] > 0:  # Short MA crossed above Long MA
                self.log(f'BUY CREATE - Price: {self.dataclose[0]:.2f}')
                # Buy with specified stake
                self.order = self.buy(size=self.params.stake)
        
        else:
            # In market - look for sell signal
            if self.crossover[0] < 0:  # Short MA crossed below Long MA
                self.log(f'SELL CREATE - Price: {self.dataclose[0]:.2f}')
                # Sell current position
                self.order = self.sell(size=self.params.stake)
    
    def stop(self):
        """Called when strategy ends"""
        self.log(f'Strategy Ended - Total Trades: {self.trade_count}', doprint=True)


class CustomCommission(bt.CommInfoBase):
    """Custom commission scheme"""
    params = (
        ('commission', 0.001),  # 0.1% commission
        ('mult', 1.0),
        ('margin', None),
    )
    
    def _getcommission(self, size, price, pseudoexec):
        return abs(size) * price * self.params.commission


def run_backtest():
    """Run the backtrader backtest"""
    
    # Create cerebro engine
    cerebro = bt.Cerebro()
    
    # Add strategy
    cerebro.addstrategy(MovingAverageCrossover,
                       short_period=10,
                       long_period=30,
                       printlog=False,  # Set to True for detailed logs
                       stake=100)
    
    # Download data using yfinance
    print("Downloading data...")
    data = yf.download('AAPL', 
                      start='2020-01-01', 
                      end='2023-12-31',
                      progress=False)
    
    # Fix column names for backtrader compatibility
    if isinstance(data.columns, pd.MultiIndex):
        # If multi-level columns, flatten them
        data.columns = [col[0] if col[1] == 'AAPL' else col[0] for col in data.columns]
    
    # Ensure proper column names
    data.columns = [col.title() for col in data.columns]
    
    # Convert to backtrader data format
    data_bt = bt.feeds.PandasData(dataname=data)
    
    # Add data to cerebro
    cerebro.adddata(data_bt)
    
    # Set initial capital
    cerebro.broker.setcash(10000.0)
    
    # Add commission
    cerebro.broker.addcommissioninfo(CustomCommission())
    
    # Add analyzers
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')
    cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
    
    # Print starting conditions
    print(f'Starting Portfolio Value: ${cerebro.broker.getvalue():.2f}')
    
    # Run backtest
    results = cerebro.run()
    strat = results[0]
    
    # Print final results
    print(f'Final Portfolio Value: ${cerebro.broker.getvalue():.2f}')
    print(f'Total Return: ${cerebro.broker.getvalue() - 10000:.2f}')
    
    # Print analyzer results
    print('\n--- PERFORMANCE METRICS ---')
    
    # Sharpe Ratio
    sharpe = strat.analyzers.sharpe.get_analysis()
    print(f'Sharpe Ratio: {sharpe.get("sharperatio", "N/A")}')
    
    # Max Drawdown
    drawdown = strat.analyzers.drawdown.get_analysis()
    print(f'Max Drawdown: {drawdown.get("max", {}).get("drawdown", "N/A"):.2f}%')
    
    # Trade Analysis
    trades = strat.analyzers.trades.get_analysis()
    total_trades = trades.get('total', {}).get('total', 0)
    won_trades = trades.get('won', {}).get('total', 0)
    lost_trades = trades.get('lost', {}).get('total', 0)
    
    print(f'Total Trades: {total_trades}')
    print(f'Winning Trades: {won_trades}')
    print(f'Losing Trades: {lost_trades}')
    
    if total_trades > 0:
        win_rate = (won_trades / total_trades) * 100
        print(f'Win Rate: {win_rate:.1f}%')
    
    # Plot results (optional - requires matplotlib)
    try:
        cerebro.plot(style='candlestick', volume=False)
    except:
        print("Plotting requires matplotlib. Install with: pip install matplotlib")
    
    return cerebro, results


if __name__ == '__main__':
    # Run the backtest
    cerebro, results = run_backtest()
    
    print("\n--- BACKTEST COMPLETED ---")
    print("This example demonstrates:")
    print("1. Creating a custom strategy with parameters")
    print("2. Using technical indicators (Moving Averages)")
    print("3. Implementing buy/sell logic")
    print("4. Adding commission and analyzers")
    print("5. Performance evaluation")
    
    # Optional: Access the strategy object for further analysis
    strategy = results[0]
    print(f"\nStrategy completed {strategy.trade_count} trades")

Starting Portfolio Value: $10000.00


C:\Users\Tolga\AppData\Local\Temp\ipykernel_16072\2483218657.py:143: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download('AAPL',


2023-12-29: Strategy Ended - Total Trades: 6
Final Portfolio Value: $12574.18
Total Return: $2574.18

--- PERFORMANCE METRICS ---
Sharpe Ratio: 0.37444083841736364
Max Drawdown: 28.02%
Total Trades: 3
Winning Trades: 2
Losing Trades: 1
Win Rate: 66.7%


<IPython.core.display.Javascript object>


--- BACKTEST COMPLETED ---
This example demonstrates:
1. Creating a custom strategy with parameters
2. Using technical indicators (Moving Averages)
3. Implementing buy/sell logic
4. Adding commission and analyzers
5. Performance evaluation

Strategy completed 6 trades


In [9]:
from backtesting import Backtest, Strategy
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_ta as ta
from datetime import datetime

class YourCustomStrategy(Strategy):
    """
    Strategy based on your technical analysis indicators
    """
    # Strategy parameters (you can optimize these)
    rsi_oversold = 30
    rsi_overbought = 70
    adx_threshold = 20
    mfi_oversold = 20
    mfi_overbought = 80
    stochrsi_oversold = 20
    stochrsi_overbought = 80
    bb_lower_threshold = 0.1  # %B threshold for oversold
    bb_upper_threshold = 0.9  # %B threshold for overbought
    
    def init(self):
        """Initialize all indicators"""
        # Convert data to pandas Series for easier calculation
        high = pd.Series(self.data.High, index=self.data.index)
        low = pd.Series(self.data.Low, index=self.data.index)
        close = pd.Series(self.data.Close, index=self.data.index)
        volume = pd.Series(self.data.Volume, index=self.data.index)
        
        # Calculate all your indicators using self.I()
        self.rsi = self.I(ta.rsi, close, length=14)
        
        # ADX - extract just the ADX column
        def get_adx(high, low, close):
            adx_data = ta.adx(high, low, close, length=14)
            return adx_data.iloc[:, 0]  # ADX column
        self.adx = self.I(get_adx, high, low, close)
        
        # MACD - extract individual components
        def get_macd_line(close):
            macd_data = ta.macd(close, fast=12, slow=26, signal=9)
            return macd_data.iloc[:, 0]  # MACD line
        
        def get_macd_signal(close):
            macd_data = ta.macd(close, fast=12, slow=26, signal=9)
            return macd_data.iloc[:, 1]  # Signal line
        
        def get_macd_hist(close):
            macd_data = ta.macd(close, fast=12, slow=26, signal=9)
            return macd_data.iloc[:, 2]  # Histogram
        
        self.macd = self.I(get_macd_line, close)
        self.macd_signal = self.I(get_macd_signal, close)
        self.macd_hist = self.I(get_macd_hist, close)
        
        # MACDAS (your custom indicator)
        def get_macdas(close):
            macd_data = ta.macd(close, fast=12, slow=26, signal=9)
            return macd_data.iloc[:, 0] - macd_data.iloc[:, 1]  # MACD - Signal
        
        def get_macdas_signal(close):
            macd_data = ta.macd(close, fast=12, slow=26, signal=9)
            macdas = macd_data.iloc[:, 0] - macd_data.iloc[:, 1]
            return macdas.ewm(span=9).mean()
        
        self.macdas = self.I(get_macdas, close)
        self.macdas_signal = self.I(get_macdas_signal, close)
        
        # Other indicators
        self.cci = self.I(ta.cci, high, low, close, length=20)
        self.roc = self.I(ta.roc, close, length=12)
        self.atr = self.I(ta.atr, high, low, close, length=14)
        self.mfi = self.I(ta.mfi, high, low, close, volume, length=14)
        self.fwma = self.I(ta.fwma, close, length=14)
        self.obv = self.I(ta.obv, close, volume)
        self.cmf = self.I(ta.cmf, high, low, close, volume)
        self.ad = self.I(ta.ad, high, low, close, volume)
        self.vwap = self.I(ta.vwap, high, low, close, volume)
        self.kama = self.I(ta.kama, close, length=14)
        
        # Supertrend
        def get_supertrend(high, low, close):
            st_data = ta.supertrend(high, low, close, length=10, multiplier=3)
            return st_data.iloc[:, 0]  # Supertrend line
        
        def get_supertrend_direction(high, low, close):
            st_data = ta.supertrend(high, low, close, length=10, multiplier=3)
            return st_data.iloc[:, 1]  # Direction
        
        self.supertrend = self.I(get_supertrend, high, low, close)
        self.supertrend_direction = self.I(get_supertrend_direction, high, low, close)
        
        # Stochastic RSI
        def get_stochrsi_k(close):
            stochrsi_data = ta.stochrsi(close, length=14, rsi_length=14, k=3, d=3)
            return stochrsi_data.iloc[:, 0]  # %K
        
        def get_stochrsi_d(close):
            stochrsi_data = ta.stochrsi(close, length=14, rsi_length=14, k=3, d=3)
            return stochrsi_data.iloc[:, 1]  # %D
        
        self.stochrsi_k = self.I(get_stochrsi_k, close)
        self.stochrsi_d = self.I(get_stochrsi_d, close)
        
        # Bollinger Bands
        def get_bb_lower(close):
            bb_data = ta.bbands(close, length=20, std=2)
            return bb_data.iloc[:, 0]  # Lower band
        
        def get_bb_middle(close):
            bb_data = ta.bbands(close, length=20, std=2)
            return bb_data.iloc[:, 1]  # Middle band
        
        def get_bb_upper(close):
            bb_data = ta.bbands(close, length=20, std=2)
            return bb_data.iloc[:, 2]  # Upper band
        
        def get_bb_percent(close):
            bb_data = ta.bbands(close, length=20, std=2)
            return bb_data.iloc[:, 4]  # %B
        
        self.bb_lower = self.I(get_bb_lower, close)
        self.bb_middle = self.I(get_bb_middle, close)
        self.bb_upper = self.I(get_bb_upper, close)
        self.bb_percent = self.I(get_bb_percent, close)
        
        # Ichimoku (simplified - just Tenkan and Kijun)
        def get_tenkan(high, low):
            return (high.rolling(9).max() + low.rolling(9).min()) / 2
        
        def get_kijun(high, low):
            return (high.rolling(26).max() + low.rolling(26).min()) / 2
        
        self.tenkan = self.I(get_tenkan, high, low)
        self.kijun = self.I(get_kijun, high, low)
    
    def next(self):
        """Main strategy logic"""
        
        # Get current values
        current_price = self.data.Close[-1]
        
        # Your trend analysis logic
        trend_way = self.get_trend_direction()
        
        # Multiple signal confirmation system based on your analysis
        buy_signals = self.get_buy_signals()
        sell_signals = self.get_sell_signals()
        
        # Position management
        if not self.position:
            # Look for buy opportunities
            if buy_signals >= 3:  # Require at least 3 confirming signals
                self.buy(size=0.95)  # Use 95% of available cash
        
        else:
            # Look for sell opportunities
            if sell_signals >= 2:  # Require at least 2 sell signals
                self.sell()
    
    def get_trend_direction(self):
        """Determine trend direction based on your logic"""
        if len(self.adx) > 0 and len(self.roc) > 0:
            if self.adx[-1] > self.adx_threshold and self.roc[-1] > 0:
                return "upper"
            elif self.adx[-1] > self.adx_threshold and self.roc[-1] <= 0:
                return "lower"
        return "no-trend"
    
    def get_buy_signals(self):
        """Count bullish signals"""
        signals = 0
        
        # RSI oversold
        if len(self.rsi) > 0 and self.rsi[-1] < self.rsi_oversold:
            signals += 1
        
        # MFI oversold
        if len(self.mfi) > 0 and self.mfi[-1] < self.mfi_oversold:
            signals += 1
        
        # StochRSI oversold
        if len(self.stochrsi_k) > 0 and self.stochrsi_k[-1] < self.stochrsi_oversold:
            signals += 1
        
        # Bollinger Bands - price near lower band
        if len(self.bb_percent) > 0 and self.bb_percent[-1] < self.bb_lower_threshold:
            signals += 1
        
        # MACDAS bullish crossover
        if (len(self.macdas) > 1 and len(self.macdas_signal) > 1 and
            self.macdas[-1] > self.macdas_signal[-1] and 
            self.macdas[-2] <= self.macdas_signal[-2]):
            signals += 1
        
        # Price above Supertrend (bullish)
        if len(self.supertrend_direction) > 0 and self.supertrend_direction[-1] == 1:
            signals += 1
        
        # Tenkan-Kijun bullish cross
        if (len(self.tenkan) > 1 and len(self.kijun) > 1 and
            self.tenkan[-1] > self.kijun[-1] and 
            self.tenkan[-2] <= self.kijun[-2]):
            signals += 1
        
        # CMF positive (buying pressure)
        if len(self.cmf) > 0 and self.cmf[-1] > 0:
            signals += 1
        
        return signals
    
    def get_sell_signals(self):
        """Count bearish signals"""
        signals = 0
        
        # RSI overbought
        if len(self.rsi) > 0 and self.rsi[-1] > self.rsi_overbought:
            signals += 1
        
        # MFI overbought
        if len(self.mfi) > 0 and self.mfi[-1] > self.mfi_overbought:
            signals += 1
        
        # StochRSI overbought
        if len(self.stochrsi_k) > 0 and self.stochrsi_k[-1] > self.stochrsi_overbought:
            signals += 1
        
        # Bollinger Bands - price near upper band
        if len(self.bb_percent) > 0 and self.bb_percent[-1] > self.bb_upper_threshold:
            signals += 1
        
        # MACDAS bearish crossover
        if (len(self.macdas) > 1 and len(self.macdas_signal) > 1 and
            self.macdas[-1] < self.macdas_signal[-1] and 
            self.macdas[-2] >= self.macdas_signal[-2]):
            signals += 1
        
        # Price below Supertrend (bearish)
        if len(self.supertrend_direction) > 0 and self.supertrend_direction[-1] == -1:
            signals += 1
        
        # CMF negative (selling pressure)
        if len(self.cmf) > 0 and self.cmf[-1] < 0:
            signals += 1
        
        return signals

class SimplifiedStrategy(Strategy):
    """Simplified version focusing on your key indicators"""
    
    def init(self):
        close = pd.Series(self.data.Close, index=self.data.index)
        high = pd.Series(self.data.High, index=self.data.index)
        low = pd.Series(self.data.Low, index=self.data.index)
        volume = pd.Series(self.data.Volume, index=self.data.index)
        
        # Key indicators from your analysis
        self.rsi = self.I(ta.rsi, close, length=14)
        self.mfi = self.I(ta.mfi, high, low, close, volume, length=14)
        
        # MACDAS (your custom indicator) - fixed version
        def get_macdas(close):
            macd_data = ta.macd(close, fast=12, slow=26, signal=9)
            return macd_data.iloc[:, 0] - macd_data.iloc[:, 1]  # MACD - Signal
        
        def get_macdas_signal(close):
            macd_data = ta.macd(close, fast=12, slow=26, signal=9)
            macdas = macd_data.iloc[:, 0] - macd_data.iloc[:, 1]
            return macdas.ewm(span=9).mean()
        
        self.macdas = self.I(get_macdas, close)
        self.macdas_signal = self.I(get_macdas_signal, close)
        
        # Supertrend
        def get_supertrend_direction(high, low, close):
            st_data = ta.supertrend(high, low, close, length=10, multiplier=3)
            return st_data.iloc[:, 1]  # Direction
        
        self.supertrend_direction = self.I(get_supertrend_direction, high, low, close)
    
    def next(self):
        # Simple strategy: Buy when multiple oversold conditions
        if not self.position:
            if (len(self.rsi) > 0 and self.rsi[-1] < 35 and 
                len(self.mfi) > 0 and self.mfi[-1] < 25 and 
                len(self.macdas) > 0 and len(self.macdas_signal) > 0 and
                self.macdas[-1] > self.macdas_signal[-1] and
                len(self.supertrend_direction) > 0 and self.supertrend_direction[-1] == 1):
                self.buy()
        
        # Sell when overbought or trend changes
        else:
            if (len(self.rsi) > 0 and self.rsi[-1] > 65 or 
                len(self.mfi) > 0 and self.mfi[-1] > 75 or 
                len(self.supertrend_direction) > 0 and self.supertrend_direction[-1] == -1):
                self.sell()

def prepare_data(symbol, start_date, end_date):
    """Download and prepare data with all your indicators"""
    print(f"Downloading {symbol} data...")
    
    # Download data
    data = yf.download(symbol, start=start_date, end=end_date, progress=False)
    
    if data.empty:
        raise ValueError(f"No data found for {symbol}")
    
    # Fix multi-level columns if needed
    if hasattr(data.columns, 'levels'):
        data.columns = data.columns.droplevel(1)
    
    print(f"Data shape: {data.shape}")
    return data

def run_backtest(symbol="AAPL", strategy=SimplifiedStrategy):
    """Run backtest with your custom strategy"""
    
    # Prepare data
    data = prepare_data(symbol, "2022-01-01", "2024-01-01")
    
    # Create and run backtest
    bt = Backtest(data, strategy, cash=10000, commission=0.002)
    
    print(f"\n🚀 Running backtest for {symbol}...")
    stats = bt.run()
    
    # Display results
    print(f"\n📊 BACKTEST RESULTS for {symbol}:")
    print("=" * 40)
    print(f"Initial Capital: $10,000")
    print(f"Final Value: ${stats['Equity Final [$]']:,.2f}")
    print(f"Total Return: {stats['Return [%]']:.2f}%")
    print(f"Buy & Hold Return: {stats['Buy & Hold Return [%]']:.2f}%")
    print(f"Max Drawdown: {stats['Max. Drawdown [%]']:.2f}%")
    print(f"Sharpe Ratio: {stats['Sharpe Ratio']:.2f}")
    print(f"Number of Trades: {stats['# Trades']}")
    print(f"Win Rate: {stats['Win Rate [%]']:.1f}%")
    
    # Show plot (requires bokeh)
    try:
        bt.plot()
    except:
        print("Install bokeh for interactive plots: pip install bokeh")
    
    return bt, stats

def test_multiple_stocks():
    """Test strategy on multiple stocks"""
    symbols = ["AAPL", "MSFT", "GOOGL", "TSLA", "NVDA"]  # Add your BIST stocks here
    results = {}
    
    for symbol in symbols:
        try:
            print(f"\n{'='*50}")
            print(f"Testing {symbol}")
            print('='*50)
            
            bt, stats = run_backtest(symbol, SimplifiedStrategy)
            results[symbol] = {
                'Return': stats['Return [%]'],
                'Sharpe': stats['Sharpe Ratio'],
                'Max_DD': stats['Max. Drawdown [%]'],
                'Trades': stats['# Trades'],
                'Win_Rate': stats['Win Rate [%]']
            }
        except Exception as e:
            print(f"Error with {symbol}: {e}")
            continue
    
    # Summary table
    if results:
        print(f"\n📈 STRATEGY PERFORMANCE SUMMARY:")
        print("=" * 70)
        print(f"{'Symbol':<8} {'Return %':<10} {'Sharpe':<8} {'Max DD %':<10} {'Trades':<8} {'Win %':<8}")
        print("-" * 70)
        
        for symbol, metrics in results.items():
            print(f"{symbol:<8} {metrics['Return']:<10.1f} {metrics['Sharpe']:<8.2f} "
                  f"{metrics['Max_DD']:<10.1f} {metrics['Trades']:<8} {metrics['Win_Rate']:<8.1f}")
    
    return results

if __name__ == "__main__":
    print("🔥 CUSTOM BACKTESTING BASED ON YOUR ANALYSIS")
    print("=" * 60)
    
    # Single stock test
    bt, stats = run_backtest("AAPL", SimplifiedStrategy)
    
    # Multiple stocks test
    print(f"\n🎯 Testing multiple stocks...")
    results = test_multiple_stocks()
    
    print(f"\n💡 TIPS:")
    print("- Adjust strategy parameters in the class definitions")
    print("- Add your BIST stock symbols to test_multiple_stocks()")
    print("- Use YourCustomStrategy for full indicator analysis")
    print("- Use SimplifiedStrategy for faster testing")
    print("- Install bokeh for interactive plots: pip install bokeh")

🔥 CUSTOM BACKTESTING BASED ON YOUR ANALYSIS
Data shape: (501, 5)

🚀 Running backtest for AAPL...


C:\Users\Tolga\AppData\Local\Temp\ipykernel_16072\3842786823.py:304: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(symbol, start=start_date, end=end_date, progress=False)
C:\Users\Tolga\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\backtesting\backtesting.py:141: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1.29802629e+10 1.29108542e+10 1.81685971e+10 1.70711296e+10
 2.96398165e+10 1.96301790e+10 1.47171622e+10 1.45867114e+10
 1.27863306e+10 1.23259715e+10 1.05691631e+10 1.47888665e+10
 1.53508221e+10 1.29767240e+10 1.25768208e+10 1.45534702e+10
 1.40405360e+10 1.58865491e+10 1.18713831e+10 1.97538884e+10
 1.55282748e+10 1.34386147e+10 1.63994209e+10 1.53068863e+10
 1.37932944e+10 1.54930867e+10 1.76065548e+10 1.33109157e+10
 1.30914718e+10 1.17569072e+10 1.10734769e+10 1.11756941e+10


📊 BACKTEST RESULTS for AAPL:
Initial Capital: $10,000
Final Value: $10,000.00
Total Return: 0.00%
Buy & Hold Return: 16.25%
Max Drawdown: -0.00%
Sharpe Ratio: nan
Number of Trades: 0
Win Rate: nan%


C:\Users\Tolga\AppData\Local\Temp\ipykernel_16072\3842786823.py:304: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(symbol, start=start_date, end=end_date, progress=False)
C:\Users\Tolga\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\backtesting\backtesting.py:141: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1.29802629e+10 1.29108542e+10 1.81685971e+10 1.70711296e+10
 2.96398165e+10 1.96301790e+10 1.47171622e+10 1.45867114e+10
 1.27863306e+10 1.23259715e+10 1.05691631e+10 1.47888665e+10
 1.53508221e+10 1.29767240e+10 1.25768208e+10 1.45534702e+10
 1.40405360e+10 1.58865491e+10 1.18713831e+10 1.97538884e+10
 1.55282748e+10 1.34386147e+10 1.63994209e+10 1.53068863e+10
 1.37932944e+10 1.54930867e+10 1.76065548e+10 1.33109157e+10
 1.30914718e+10 1.17569072e+10 1.10734769e+10 1.11756941e+10

Install bokeh for interactive plots: pip install bokeh

🎯 Testing multiple stocks...

Testing AAPL
Data shape: (501, 5)

🚀 Running backtest for AAPL...


...

Data shape: (501, 5)

🚀 Running backtest for TSLA...



📊 BACKTEST RESULTS for TSLA:
Initial Capital: $10,000
Final Value: $10,000.00
Total Return: 0.00%
Buy & Hold Return: -13.02%
Max Drawdown: -0.00%
Sharpe Ratio: nan
Number of Trades: 0
Win Rate: nan%
Install bokeh for interactive plots: pip install bokeh

Testing NVDA


C:\Users\Tolga\AppData\Local\Temp\ipykernel_16072\3842786823.py:304: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(symbol, start=start_date, end=end_date, progress=False)
C:\Users\Tolga\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\backtesting\backtesting.py:141: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1.11219191e+10 1.07417784e+10 1.73878316e+10 1.35364619e+10
 1.27203748e+10 1.36713854e+10 9.29318858e+09 1.01471697e+10
 1.37430921e+10 1.81224669e+10 1.91721140e+10 1.26265546e+10
 1.15550125e+10 9.34167040e+09 1.12292466e+10 1.10242859e+10
 1.61467282e+10 1.15456585e+10 1.88812374e+10 1.57243822e+10
 2.40521618e+10 1.60548404e+10 1.18481369e+10 1.39263711e+10
 1.07747074e+10 1.13811452e+10 1.12429281e+10 1.10667581e+10
 1.09457854e+10 9.27132127e+09 1.28444669e+10 1.33843728e+10

Data shape: (501, 5)

🚀 Running backtest for NVDA...



📊 BACKTEST RESULTS for NVDA:
Initial Capital: $10,000
Final Value: $10,000.00
Total Return: 0.00%
Buy & Hold Return: 109.76%
Max Drawdown: -0.00%
Sharpe Ratio: nan
Number of Trades: 0
Win Rate: nan%
Install bokeh for interactive plots: pip install bokeh

📈 STRATEGY PERFORMANCE SUMMARY:
Symbol   Return %   Sharpe   Max DD %   Trades   Win %   
----------------------------------------------------------------------
AAPL     0.0        nan      -0.0       0        nan     
MSFT     0.0        nan      -0.0       0        nan     
GOOGL    0.0        nan      -0.0       0        nan     
TSLA     0.0        nan      -0.0       0        nan     
NVDA     0.0        nan      -0.0       0        nan     

💡 TIPS:
- Adjust strategy parameters in the class definitions
- Add your BIST stock symbols to test_multiple_stocks()
- Use YourCustomStrategy for full indicator analysis
- Use SimplifiedStrategy for faster testing
- Install bokeh for interactive plots: pip install bokeh


In [16]:
from backtesting import Backtest, Strategy
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_ta as ta
from datetime import datetime
from abc import ABC, abstractmethod

# =============================================================================
# BASE STRATEGY CLASS (Common functionality)
# =============================================================================

class BaseStrategy(Strategy):
    """Base strategy class with common indicators and utilities"""
    
    def __init__(self, broker, data, params):
        super().__init__(broker, data, params)
        # Common parameters that all strategies can use
        self.rsi_period = getattr(self, 'rsi_period', 14)
        self.ma_period = getattr(self, 'ma_period', 20)
        self.atr_period = getattr(self, 'atr_period', 14)
        # Position sizing parameters
        self.position_size = getattr(self, 'position_size', 0.95)  # Use 95% of available cash
        self.min_cash_reserve = getattr(self, 'min_cash_reserve', 100)  # Keep minimum cash
    
    def setup_common_indicators(self):
        """Setup indicators used across multiple strategies"""
        close = pd.Series(self.data.Close, index=self.data.index)
        high = pd.Series(self.data.High, index=self.data.index)
        low = pd.Series(self.data.Low, index=self.data.index)
        volume = pd.Series(self.data.Volume, index=self.data.index)
        
        # Basic indicators
        self.rsi = self.I(ta.rsi, close, length=self.rsi_period)
        self.sma = self.I(ta.sma, close, length=self.ma_period)
        self.atr = self.I(ta.atr, high, low, close, length=self.atr_period)
        
        return close, high, low, volume
    
    def is_valid_signal(self, *indicators):
        """Check if all indicators have valid values"""
        return all(len(ind) > 0 for ind in indicators if ind is not None)
    
    def safe_buy(self, size=None):
        """Safe buy with proper position sizing"""
        if size is None:
            # Use fractional sizing (percentage of equity)
            size = self.position_size  # This will be 0.95 (95% of equity)
        
        try:
            self.buy(size=size)
        except Exception:
            # If buy fails, try with smaller fractional size
            try:
                smaller_size = size * 0.5
                if smaller_size > 0.1:  # At least 10% of equity
                    self.buy(size=smaller_size)
            except Exception:
                pass  # Skip this trade if still fails
    
    def safe_sell(self, size=None):
        """Safe sell method"""
        try:
            if size is None:
                self.sell()  # Sell entire position
            else:
                self.sell(size=size)
        except Exception:
            pass  # Skip if sell fails

# =============================================================================
# STRATEGY CATEGORIES
# =============================================================================

# 1. TREND FOLLOWING STRATEGIES
class TrendFollowingStrategy(BaseStrategy):
    """Strategy focused on trend following"""
    
    # Strategy parameters
    adx_threshold = 25
    ma_fast = 10
    ma_slow = 30
    
    def init(self):
        close, high, low, volume = self.setup_common_indicators()
        
        # Trend indicators
        self.ma_fast = self.I(ta.sma, close, length=self.ma_fast)
        self.ma_slow = self.I(ta.sma, close, length=self.ma_slow)
        
        def get_adx(high, low, close):
            adx_data = ta.adx(high, low, close, length=14)
            return adx_data.iloc[:, 0] if len(adx_data.columns) > 0 else pd.Series([np.nan] * len(close))
        self.adx = self.I(get_adx, high, low, close)
        
        def get_supertrend_direction(high, low, close):
            try:
                st_data = ta.supertrend(high, low, close, length=10, multiplier=3)
                return st_data.iloc[:, 1] if len(st_data.columns) > 1 else pd.Series([1] * len(close))
            except:
                return pd.Series([1] * len(close))
        self.supertrend_dir = self.I(get_supertrend_direction, high, low, close)
    
    def next(self):
        if not self.position:
            # Buy when fast MA crosses above slow MA and strong trend
            if (self.is_valid_signal(self.ma_fast, self.ma_slow, self.adx, self.supertrend_dir) and
                self.ma_fast[-1] > self.ma_slow[-1] and
                self.adx[-1] > self.adx_threshold and
                self.supertrend_dir[-1] == 1):
                self.safe_buy()
        else:
            # Sell when trend reverses
            if (self.is_valid_signal(self.ma_fast, self.ma_slow, self.supertrend_dir) and
                (self.ma_fast[-1] < self.ma_slow[-1] or self.supertrend_dir[-1] == -1)):
                self.safe_sell()

class MomentumStrategy(BaseStrategy):
    """Strategy focused on momentum indicators"""
    
    # Strategy parameters
    rsi_oversold = 30
    rsi_overbought = 70
    stoch_oversold = 20
    stoch_overbought = 80
    
    def init(self):
        close, high, low, volume = self.setup_common_indicators()
        
        # Momentum indicators
        def get_stoch_k(high, low, close):
            try:
                stoch_data = ta.stoch(high, low, close, k=14, d=3)
                return stoch_data.iloc[:, 0] if len(stoch_data.columns) > 0 else pd.Series([50] * len(close))
            except:
                return pd.Series([50] * len(close))
        
        def get_stoch_d(high, low, close):
            try:
                stoch_data = ta.stoch(high, low, close, k=14, d=3)
                return stoch_data.iloc[:, 1] if len(stoch_data.columns) > 1 else pd.Series([50] * len(close))
            except:
                return pd.Series([50] * len(close))
        
        self.stoch_k = self.I(get_stoch_k, high, low, close)
        self.stoch_d = self.I(get_stoch_d, high, low, close)
        self.mfi = self.I(ta.mfi, high, low, close, volume, length=14)
    
    def next(self):
        if not self.position:
            # Buy on oversold conditions
            if (self.is_valid_signal(self.rsi, self.stoch_k, self.mfi) and
                self.rsi[-1] < self.rsi_oversold and
                self.stoch_k[-1] < self.stoch_oversold and
                self.mfi[-1] < 25):
                self.safe_buy()
        else:
            # Sell on overbought conditions
            if (self.is_valid_signal(self.rsi, self.stoch_k, self.mfi) and
                (self.rsi[-1] > self.rsi_overbought or
                 self.stoch_k[-1] > self.stoch_overbought or
                 self.mfi[-1] > 75)):
                self.safe_sell()

class MeanReversionStrategy(BaseStrategy):
    """Strategy focused on mean reversion"""
    
    # Strategy parameters
    bb_std = 2
    bb_period = 20
    rsi_extreme_oversold = 25
    rsi_extreme_overbought = 75
    
    def init(self):
        close, high, low, volume = self.setup_common_indicators()
        
        # Mean reversion indicators
        def get_bb_lower(close):
            try:
                bb_data = ta.bbands(close, length=self.bb_period, std=self.bb_std)
                return bb_data.iloc[:, 0] if len(bb_data.columns) > 0 else close * 0.98
            except:
                return close * 0.98
        
        def get_bb_upper(close):
            try:
                bb_data = ta.bbands(close, length=self.bb_period, std=self.bb_std)
                return bb_data.iloc[:, 2] if len(bb_data.columns) > 2 else close * 1.02
            except:
                return close * 1.02
        
        def get_bb_percent(close):
            try:
                bb_data = ta.bbands(close, length=self.bb_period, std=self.bb_std)
                return bb_data.iloc[:, 4] if len(bb_data.columns) > 4 else pd.Series([0.5] * len(close))
            except:
                return pd.Series([0.5] * len(close))
        
        self.bb_lower = self.I(get_bb_lower, close)
        self.bb_upper = self.I(get_bb_upper, close)
        self.bb_percent = self.I(get_bb_percent, close)
    
    def next(self):
        if not self.position:
            # Buy when price touches lower Bollinger Band and RSI oversold
            if (self.is_valid_signal(self.bb_percent, self.rsi) and
                self.bb_percent[-1] < 0.1 and
                self.rsi[-1] < self.rsi_extreme_oversold):
                self.safe_buy()
        else:
            # Sell when price touches upper Bollinger Band or RSI overbought
            if (self.is_valid_signal(self.bb_percent, self.rsi) and
                (self.bb_percent[-1] > 0.9 or self.rsi[-1] > self.rsi_extreme_overbought)):
                self.safe_sell()

# 2. COMPLEX MULTI-SIGNAL STRATEGIES
class ConservativeStrategy(BaseStrategy):
    """Conservative strategy requiring multiple confirmations"""
    
    min_buy_signals = 4
    min_sell_signals = 2
    
    def init(self):
        close, high, low, volume = self.setup_common_indicators()
        
        # Multiple indicators for confirmation
        self.mfi = self.I(ta.mfi, high, low, close, volume, length=14)
        self.cci = self.I(ta.cci, high, low, close, length=20)
        
        def get_macd_bullish(close):
            try:
                macd_data = ta.macd(close, fast=12, slow=26, signal=9)
                if len(macd_data.columns) >= 2:
                    return macd_data.iloc[:, 0] > macd_data.iloc[:, 1]  # MACD > Signal
                else:
                    return pd.Series([False] * len(close))
            except:
                return pd.Series([False] * len(close))
        
        self.macd_bullish = self.I(get_macd_bullish, close)
        
        def get_bb_percent(close):
            try:
                bb_data = ta.bbands(close, length=20, std=2)
                return bb_data.iloc[:, 4] if len(bb_data.columns) > 4 else pd.Series([0.5] * len(close))
            except:
                return pd.Series([0.5] * len(close))
        
        self.bb_percent = self.I(get_bb_percent, close)
    
    def count_buy_signals(self):
        """Count bullish signals"""
        signals = 0
        if self.is_valid_signal(self.rsi) and self.rsi[-1] < 35:
            signals += 1
        if self.is_valid_signal(self.mfi) and self.mfi[-1] < 30:
            signals += 1
        if self.is_valid_signal(self.cci) and self.cci[-1] < -100:
            signals += 1
        if self.is_valid_signal(self.macd_bullish) and self.macd_bullish[-1]:
            signals += 1
        if self.is_valid_signal(self.bb_percent) and self.bb_percent[-1] < 0.2:
            signals += 1
        return signals
    
    def count_sell_signals(self):
        """Count bearish signals"""
        signals = 0
        if self.is_valid_signal(self.rsi) and self.rsi[-1] > 65:
            signals += 1
        if self.is_valid_signal(self.mfi) and self.mfi[-1] > 70:
            signals += 1
        if self.is_valid_signal(self.cci) and self.cci[-1] > 100:
            signals += 1
        if self.is_valid_signal(self.bb_percent) and self.bb_percent[-1] > 0.8:
            signals += 1
        return signals
    
    def next(self):
        if not self.position:
            if self.count_buy_signals() >= self.min_buy_signals:
                self.safe_buy()
        else:
            if self.count_sell_signals() >= self.min_sell_signals:
                self.safe_sell()

class AggressiveStrategy(BaseStrategy):
    """Aggressive strategy with quick entries/exits"""
    
    def init(self):
        close, high, low, volume = self.setup_common_indicators()
        
        # Fast indicators
        self.rsi_fast = self.I(ta.rsi, close, length=9)
        self.ema_fast = self.I(ta.ema, close, length=8)
        self.ema_slow = self.I(ta.ema, close, length=21)
    
    def next(self):
        if not self.position:
            # Quick entry on fast EMA crossover and RSI recovery
            if (self.is_valid_signal(self.ema_fast, self.ema_slow, self.rsi_fast) and
                self.ema_fast[-1] > self.ema_slow[-1] and
                self.rsi_fast[-1] > 40 and self.rsi_fast[-1] < 60):
                self.safe_buy()
        else:
            # Quick exit on reversal
            if (self.is_valid_signal(self.ema_fast, self.ema_slow, self.rsi_fast) and
                (self.ema_fast[-1] < self.ema_slow[-1] or 
                 self.rsi_fast[-1] < 35 or self.rsi_fast[-1] > 70)):
                self.safe_sell()

# 3. YOUR CUSTOM STRATEGIES
class YourCustomMACDAS(BaseStrategy):
    """Your custom MACDAS strategy"""
    
    def init(self):
        close, high, low, volume = self.setup_common_indicators()
        
        # Your custom MACDAS indicator
        def get_macdas(close):
            try:
                macd_data = ta.macd(close, fast=12, slow=26, signal=9)
                if len(macd_data.columns) >= 2:
                    return macd_data.iloc[:, 0] - macd_data.iloc[:, 1]
                else:
                    return pd.Series([0] * len(close))
            except:
                return pd.Series([0] * len(close))
        
        def get_macdas_signal(close):
            try:
                macd_data = ta.macd(close, fast=12, slow=26, signal=9)
                if len(macd_data.columns) >= 2:
                    macdas = macd_data.iloc[:, 0] - macd_data.iloc[:, 1]
                    return macdas.ewm(span=9).mean()
                else:
                    return pd.Series([0] * len(close))
            except:
                return pd.Series([0] * len(close))
        
        self.macdas = self.I(get_macdas, close)
        self.macdas_signal = self.I(get_macdas_signal, close)
        
        # Additional indicators
        self.mfi = self.I(ta.mfi, high, low, close, volume, length=14)
        
        def get_supertrend_direction(high, low, close):
            try:
                st_data = ta.supertrend(high, low, close, length=10, multiplier=3)
                return st_data.iloc[:, 1] if len(st_data.columns) > 1 else pd.Series([1] * len(close))
            except:
                return pd.Series([1] * len(close))
        
        self.supertrend_dir = self.I(get_supertrend_direction, high, low, close)
    
    def next(self):
        if not self.position:
            # Your custom entry logic
            if (self.is_valid_signal(self.macdas, self.macdas_signal, self.rsi, self.mfi, self.supertrend_dir) and
                self.macdas[-1] > self.macdas_signal[-1] and
                self.rsi[-1] < 40 and
                self.mfi[-1] < 30 and
                self.supertrend_dir[-1] == 1):
                self.safe_buy()
        else:
            # Your custom exit logic
            if (self.is_valid_signal(self.macdas, self.macdas_signal, self.rsi, self.supertrend_dir) and
                (self.macdas[-1] < self.macdas_signal[-1] or
                 self.rsi[-1] > 70 or
                 self.supertrend_dir[-1] == -1)):
                self.safe_sell()

# =============================================================================
# STRATEGY MANAGER AND TESTING FRAMEWORK
# =============================================================================

class StrategyManager:
    """Manage and organize multiple strategies"""
    
    def __init__(self):
        self.strategies = {
            # Trend Following
            'trend_following': TrendFollowingStrategy,
            
            # Momentum
            'momentum': MomentumStrategy,
            
            # Mean Reversion
            'mean_reversion': MeanReversionStrategy,
            
            # Multi-Signal
            'conservative': ConservativeStrategy,
            'aggressive': AggressiveStrategy,
            
            # Custom
            'macdas_custom': YourCustomMACDAS,
        }
        
        self.results = {}
    
    def list_strategies(self):
        """List all available strategies"""
        print("📋 AVAILABLE STRATEGIES:")
        print("=" * 40)
        for name, strategy_class in self.strategies.items():
            print(f"• {name}: {strategy_class.__doc__ or 'No description'}")
    
    def get_strategy(self, name):
        """Get strategy class by name"""
        if name not in self.strategies:
            raise ValueError(f"Strategy '{name}' not found. Available: {list(self.strategies.keys())}")
        return self.strategies[name]
    
    def add_custom_strategy(self, name, strategy_class):
        """Add a custom strategy"""
        self.strategies[name] = strategy_class
        print(f"✅ Added custom strategy: {name}")
    
    def run_single_strategy(self, strategy_name, symbol="AAPL", start_date="2022-01-01", end_date="2024-01-01", cash=10000):
        """Run a single strategy"""
        strategy_class = self.get_strategy(strategy_name)
        
        # Get data
        data = self.prepare_data(symbol, start_date, end_date)
        
        # Run backtest with proper configuration
        bt = Backtest(
            data, 
            strategy_class, 
            cash=cash, 
            commission=0.002,
            margin=1.0,  # No leverage
            trade_on_close=False,  # Trade on next open (more realistic)
            hedging=False,  # Disable hedging to avoid position conflicts
            exclusive_orders=True  # Prevent conflicting orders
        )
        
        try:
            stats = bt.run()
            
            # Store results
            self.results[f"{strategy_name}_{symbol}"] = stats
            
            # Display results
            self.display_results(strategy_name, symbol, stats)
            
            return bt, stats
        except Exception as e:
            print(f"❌ Error running {strategy_name} on {symbol}: {e}")
            print(f"   Data shape: {data.shape if data is not None else 'None'}")
            print(f"   Data columns: {list(data.columns) if data is not None else 'None'}")
            return None, None
    
    def run_strategy_comparison(self, strategies, symbol="AAPL", start_date="2022-01-01", end_date="2024-01-01"):
        """Compare multiple strategies on the same symbol"""
        results = {}
        
        print(f"\n🔥 STRATEGY COMPARISON: {symbol}")
        print("=" * 60)
        
        for strategy_name in strategies:
            try:
                print(f"\n🚀 Testing {strategy_name}...")
                bt, stats = self.run_single_strategy(strategy_name, symbol, start_date, end_date)
                if stats is not None:
                    results[strategy_name] = stats
            except Exception as e:
                print(f"❌ Error with {strategy_name}: {e}")
        
        # Comparison table
        if results:
            self.display_comparison(results, symbol)
        return results
    
    def run_multi_asset_test(self, strategy_name, symbols, start_date="2022-01-01", end_date="2024-01-01"):
        """Test one strategy across multiple assets"""
        results = {}
        
        print(f"\n🎯 MULTI-ASSET TEST: {strategy_name}")
        print("=" * 60)
        
        for symbol in symbols:
            try:
                print(f"\n📈 Testing {symbol}...")
                bt, stats = self.run_single_strategy(strategy_name, symbol, start_date, end_date)
                if stats is not None:
                    results[symbol] = stats
            except Exception as e:
                print(f"❌ Error with {symbol}: {e}")
        
        # Multi-asset comparison
        if results:
            self.display_multi_asset_results(results, strategy_name)
        return results
    
    @staticmethod
    def prepare_data(symbol, start_date, end_date):
        """Prepare data for backtesting"""
        try:
            # Fix the yfinance warning
            data = yf.download(
                symbol, 
                start=start_date, 
                end=end_date, 
                progress=False,
                auto_adjust=True  # Explicitly set to avoid warning
            )
            if hasattr(data.columns, 'levels'):
                data.columns = data.columns.droplevel(1)
            
            # Ensure we have enough data
            if len(data) < 50:
                raise ValueError(f"Insufficient data for {symbol}: only {len(data)} rows")
            
            # Remove any NaN values
            data = data.dropna()
            
            return data
        except Exception as e:
            print(f"❌ Error preparing data for {symbol}: {e}")
            raise
    
    @staticmethod
    def display_results(strategy_name, symbol, stats):
        """Display individual strategy results"""
        print(f"\n📊 {strategy_name.upper()} RESULTS for {symbol}:")
        print("-" * 50)
        print(f"Return: {stats['Return [%]']:.2f}%")
        print(f"Buy & Hold: {stats['Buy & Hold Return [%]']:.2f}%")
        print(f"Sharpe Ratio: {stats['Sharpe Ratio']:.2f}")
        print(f"Max Drawdown: {stats['Max. Drawdown [%]']:.2f}%")
        print(f"Trades: {stats['# Trades']}")
        print(f"Win Rate: {stats['Win Rate [%]']:.1f}%")
    
    @staticmethod
    def display_comparison(results, symbol):
        """Display strategy comparison table"""
        if not results:
            return
        
        print(f"\n📈 STRATEGY COMPARISON SUMMARY: {symbol}")
        print("=" * 80)
        print(f"{'Strategy':<15} {'Return %':<10} {'Sharpe':<8} {'Max DD %':<10} {'Trades':<8} {'Win %':<8}")
        print("-" * 80)
        
        for strategy, stats in results.items():
            print(f"{strategy:<15} {stats['Return [%]']:<10.1f} {stats['Sharpe Ratio']:<8.2f} "
                  f"{stats['Max. Drawdown [%]']:<10.1f} {stats['# Trades']:<8} {stats['Win Rate [%]']:<8.1f}")
    
    @staticmethod
    def display_multi_asset_results(results, strategy_name):
        """Display multi-asset test results"""
        if not results:
            return
        
        print(f"\n🌍 MULTI-ASSET RESULTS: {strategy_name}")
        print("=" * 70)
        print(f"{'Symbol':<8} {'Return %':<10} {'Sharpe':<8} {'Max DD %':<10} {'Trades':<8} {'Win %':<8}")
        print("-" * 70)
        
        for symbol, stats in results.items():
            print(f"{symbol:<8} {stats['Return [%]']:<10.1f} {stats['Sharpe Ratio']:<8.2f} "
                  f"{stats['Max. Drawdown [%]']:<10.1f} {stats['# Trades']:<8} {stats['Win Rate [%]']:<8.1f}")

# =============================================================================
# USAGE EXAMPLES
# =============================================================================

def main():
    """Main function showing how to use the organized strategies"""
    
    # Create strategy manager
    manager = StrategyManager()
    
    # List available strategies
    manager.list_strategies()
    
    # Example 1: Run a single strategy
    print("\n" + "="*60)
    print("EXAMPLE 1: Single Strategy Test")
    print("="*60)
    bt, stats = manager.run_single_strategy('trend_following', 'AAPL')
    
    # Example 2: Compare multiple strategies
    print("\n" + "="*60)
    print("EXAMPLE 2: Strategy Comparison")
    print("="*60)
    strategies_to_compare = ['trend_following', 'momentum', 'conservative']
    comparison_results = manager.run_strategy_comparison(strategies_to_compare, 'AAPL')
    
    # Example 3: Test one strategy across multiple stocks
    print("\n" + "="*60)
    print("EXAMPLE 3: Multi-Asset Test")
    print("="*60)
    symbols = ['AAPL', 'MSFT', 'GOOGL']
    multi_asset_results = manager.run_multi_asset_test('macdas_custom', symbols)
    
    # Example 4: Add your own custom strategy
    print("\n" + "="*60)
    print("EXAMPLE 4: Adding Custom Strategy")
    print("="*60)
    
    class MyPersonalStrategy(BaseStrategy):
        """My personal trading strategy"""
        def init(self):
            close, high, low, volume = self.setup_common_indicators()
            self.ema = self.I(ta.ema, close, length=12)
        
        def next(self):
            if not self.position and self.data.Close[-1] > self.ema[-1]:
                self.safe_buy()
            elif self.position and self.data.Close[-1] < self.ema[-1]:
                self.safe_sell()
    
    manager.add_custom_strategy('my_personal', MyPersonalStrategy)
    bt, stats = manager.run_single_strategy('my_personal', 'AAPL')

if __name__ == "__main__":
    main()

📋 AVAILABLE STRATEGIES:
• trend_following: Strategy focused on trend following
• momentum: Strategy focused on momentum indicators
• mean_reversion: Strategy focused on mean reversion
• conservative: Conservative strategy requiring multiple confirmations
• aggressive: Aggressive strategy with quick entries/exits
• macdas_custom: Your custom MACDAS strategy

EXAMPLE 1: Single Strategy Test



📊 TREND_FOLLOWING RESULTS for AAPL:
--------------------------------------------------
Return: -72.38%
Buy & Hold: 15.16%
Sharpe Ratio: -3.10
Max Drawdown: -73.53%
Trades: 224
Win Rate: 49.1%

EXAMPLE 2: Strategy Comparison

🔥 STRATEGY COMPARISON: AAPL

🚀 Testing trend_following...



📊 TREND_FOLLOWING RESULTS for AAPL:
--------------------------------------------------
Return: -72.38%
Buy & Hold: 15.16%
Sharpe Ratio: -3.10
Max Drawdown: -73.53%
Trades: 224
Win Rate: 49.1%

🚀 Testing momentum...
❌ Error running momentum on AAPL: Indicators must return (optionally a tuple of) numpy.arrays of same length as `data` (data shape: (501,); indicator "get_stoch…" shape: (488,), returned value: [        nan         nan 14.99403799 22.2155376  20.74972812 37.27754611
 59.65296443 82.47323648 91.44430277 89.83436923 88.02713322 82.46411989
 85.79555004 90.85669567 90.78773729 79.16042329 66.10779982 65.99769837
 69.74009989 57.6806426  34.81280498 16.19017247  9.15697975 20.14302947
 32.53112457 49.64163483 50.2636553  53.55706527 58.04278333 60.34506062
 51.07769223 37.34762971 38.23246965 38.55217295 35.43625718 18.02526323
 15.14568691 26.58156115 44.30266423 60.05673689 70.31728878 83.97286258
 89.11574071 95.27417025 95.56802506 98.9925656  99.0319788  97.69491728
 92.21


📊 CONSERVATIVE RESULTS for AAPL:
--------------------------------------------------
Return: -51.74%
Buy & Hold: 11.41%
Sharpe Ratio: -1.77
Max Drawdown: -62.57%
Trades: 147
Win Rate: 35.4%

📈 STRATEGY COMPARISON SUMMARY: AAPL
Strategy        Return %   Sharpe   Max DD %   Trades   Win %   
--------------------------------------------------------------------------------
trend_following -72.4      -3.10    -73.5      224      49.1    
conservative    -51.7      -1.77    -62.6      147      35.4    

EXAMPLE 3: Multi-Asset Test

🎯 MULTI-ASSET TEST: macdas_custom

📈 Testing AAPL...



📊 MACDAS_CUSTOM RESULTS for AAPL:
--------------------------------------------------
Return: 0.00%
Buy & Hold: 16.25%
Sharpe Ratio: nan
Max Drawdown: -0.00%
Trades: 0
Win Rate: nan%

📈 Testing MSFT...



📊 MACDAS_CUSTOM RESULTS for MSFT:
--------------------------------------------------
Return: 0.00%
Buy & Hold: 32.72%
Sharpe Ratio: nan
Max Drawdown: -0.00%
Trades: 0
Win Rate: nan%

📈 Testing GOOGL...



📊 MACDAS_CUSTOM RESULTS for GOOGL:
--------------------------------------------------
Return: 0.00%
Buy & Hold: 7.12%
Sharpe Ratio: nan
Max Drawdown: -0.00%
Trades: 0
Win Rate: nan%

🌍 MULTI-ASSET RESULTS: macdas_custom
Symbol   Return %   Sharpe   Max DD %   Trades   Win %   
----------------------------------------------------------------------
AAPL     0.0        nan      -0.0       0        nan     
MSFT     0.0        nan      -0.0       0        nan     
GOOGL    0.0        nan      -0.0       0        nan     

EXAMPLE 4: Adding Custom Strategy
✅ Added custom strategy: my_personal



📊 MY_PERSONAL RESULTS for AAPL:
--------------------------------------------------
Return: -70.44%
Buy & Hold: 11.41%
Sharpe Ratio: -2.77
Max Drawdown: -70.88%
Trades: 206
Win Rate: 52.4%


# Old Version Starts Here

In [1]:
#Import part
import pandas as pd
import yfinance as yf
import numpy as np
import pandas_ta as ta
import time
import glob
from datetime import datetime, timedelta

In [ ]:

#In this section, we will try to find out correct TA numbers for strategy development.
# we can make a new row for each day and complete as a dataframe
# <<<================= Just update here ==========================>>>
bist1 = ["BIMAS.IS"]
endDate   = "2025-07-18"
startDate = "2024-05-01"
# <<<================= Just update the upper part ================>>> 

def date_based_test(startDate,endDate,bist1):
    stock_dict = {}
    stock_data = []
    stock_number = 0

    #Data has to download one time & we have to use it part by part
    for stocks in bist1:
        try:
            ticker = yf.Ticker(stocks)
            stock_info = ticker.info
            data = {key: stock_info.get(key, None) for key in ["symbol", "priceToBook", "currentPrice", "targetHighPrice", "targetLowPrice", "targetMeanPrice", "targetMedianPrice",
                                                                "bookValue", "open", "dayLow", "dayHigh", "recommendationKey", 'fiftyTwoWeekLow', 'fiftyTwoWeekHigh']}
            #historical_data = ticker.history(period="1y")  # Get historical data for the stock
            #Date Test for spesific scores
            historical_data = ticker.history(start=startDate, end=endDate)  # Get historical data for the stock
            stock_dict[stocks] = historical_data
            if historical_data.empty:
                print(f"No historical data for {stocks}, skipping...")
                continue  # Skip this ticker if no historical data is found
            stock_number = 1 + stock_number
            #print(f"\r{stock_number}/{len(bist1)} Downloaded {stocks} data", end='', flush=True)

            historical_data['RSI'] = ta.rsi(historical_data['Close'], length=14)  # Calculate RSI (Relative Strength Index)
            adx_data = ta.adx(historical_data['High'], historical_data['Low'], historical_data['Close'], length=14)  # Calculate ADX (Average Directional Index)
            historical_data['ADX'] = adx_data['ADX_14']

            # Calculate MACD using EMA
            macd = ta.macd(historical_data['Close'], fast=12, slow=26, signal=9)
            historical_data = pd.concat([historical_data, macd], axis=1)

            # Calculate MACDAS (MACD Histogram)
            historical_data['MACDAS'] = historical_data['MACD_12_26_9'] - historical_data['MACDs_12_26_9']

            # Calculate MACDAS Signal Line (9-period EMA of MACDAS)
            historical_data['MACDAS_Signal'] = historical_data['MACDAS'].ewm(span=9, adjust=False).mean()

            historical_data['CCI'] = ta.cci(historical_data['High'], historical_data['Low'], historical_data['Close'], length=20)  # Calculate CCI (Commodity Channel Index)
            historical_data['ROC'] = ta.roc(historical_data['Close'], length=12)  # Calculate ROC (Rate of Change)
            historical_data['ATR'] = ta.atr(historical_data['High'], historical_data['Low'], historical_data['Close'], length=14)  # Calculate ATR (Average True Range)

            # Calculate Bollinger Bands
            bollinger = ta.bbands(historical_data['Close'], length=20, std=2)
            historical_data['BB_Middle'] = bollinger.iloc[:, 1]  # Middle Band
            historical_data['BB_Upper'] = bollinger.iloc[:, 2]  # Upper Band
            historical_data['BB_Lower'] = bollinger.iloc[:, 0]  # Lower Band
            historical_data['BB_BWidth'] = bollinger.iloc[:, 3]  # Bandwidth
            historical_data['BB_%B'] = bollinger.iloc[:, 4]  # %B

            # Calculate BB Signal
            recent_close = historical_data['Close'].iloc[-1]
            recent_bb_upper = historical_data['BB_Upper'].iloc[-1]
            recent_bb_lower = historical_data['BB_Lower'].iloc[-1]

            if recent_close > recent_bb_upper:
                bb_signal = "Sell"
            elif recent_close < recent_bb_lower:
                bb_signal = "Buy"
            else:
                bb_signal = "Neutral"

            data["BB_Signal"] = bb_signal

            historical_data['OBV'] = ta.obv(historical_data['Close'], historical_data['Volume'])  # Calculate OBV (On-Balance Volume)
            historical_data['CMF'] = ta.cmf(historical_data['High'], historical_data['Low'], historical_data['Close'], historical_data['Volume'], length=20)  # Calculate CMF (Chaikin Money Flow)
            historical_data['AD'] = ta.ad(historical_data['High'], historical_data['Low'], historical_data['Close'], historical_data['Volume'])  # Calculate A/D Line (Accumulation/Distribution Line)

            high_9 = historical_data['High'].rolling(window=9).max()  # Calculate Ichimoku Cloud components
            low_9 = historical_data['Low'].rolling(window=9).min()
            historical_data['Tenkan_sen'] = (high_9 + low_9) / 2
            high_26 = historical_data['High'].rolling(window=26).max()
            low_26 = historical_data['Low'].rolling(window=26).min()
            historical_data['Kijun_sen'] = (high_26 + low_26) / 2
            historical_data['Senkou_Span_A'] = ((historical_data['Tenkan_sen'] + historical_data['Kijun_sen']) / 2).shift(26)
            high_52 = historical_data['High'].rolling(window=52).max()
            low_52 = historical_data['Low'].rolling(window=52).min()
            historical_data['Senkou_Span_B'] = ((high_52 + low_52) / 2).shift(26)
            historical_data['Chikou_Span'] = historical_data['Close'].shift(-26)

            # Buy/Sell Signal Calculation for Ichimoku
            recent_close = historical_data['Close'].iloc[-1]
            recent_tenkan = historical_data['Tenkan_sen'].iloc[-1]
            recent_kijun = historical_data['Kijun_sen'].iloc[-1]
            recent_senkou_a = historical_data['Senkou_Span_A'].iloc[-1]
            recent_senkou_b = historical_data['Senkou_Span_B'].iloc[-1]

            
            if recent_close > max(recent_senkou_a, recent_senkou_b) and recent_tenkan > recent_kijun:
                data["IchiSignal"] = "Buy"
            elif recent_close < min(recent_senkou_a, recent_senkou_b) and recent_tenkan < recent_kijun:
                data["IchiSignal"] = "Sell"
            else:
                data["IchiSignal"] = "Neutral"

            data.update({"Tenkan_sen": recent_tenkan, "Kijun_sen": recent_kijun, "Senkou_Span_A": recent_senkou_a, "Senkou_Span_B": recent_senkou_b,
                         "Chikou_Span": historical_data['Chikou_Span'].iloc[-1], "RSI": historical_data['RSI'].iloc[-1], "ADX": historical_data['ADX'].iloc[-1],
                         "CCI": historical_data['CCI'].iloc[-1], "ROC": historical_data['ROC'].iloc[-1], "ATR": historical_data['ATR'].iloc[-1], "OBV": historical_data['OBV'].iloc[-1],
                         "CMF": historical_data['CMF'].iloc[-1], "AD": historical_data['AD'].iloc[-1],
                         "MACD": historical_data['MACD_12_26_9'].iloc[-1],  # MACD Line
                         "MACD_signal": historical_data['MACDs_12_26_9'].iloc[-1],  # MACD Signal Line
                         "MACD_Hist": historical_data['MACDh_12_26_9'].iloc[-1],  # MACD Histogram
                         "MACDAS": historical_data['MACDAS'].iloc[-1],  # MACDAS
                         "MACDAS_Signal": historical_data['MACDAS_Signal'].iloc[-1],  # MACDAS Signal Line
                         "BB_Middle": historical_data['BB_Middle'].iloc[-1],
                         "BB_Upper": historical_data['BB_Upper'].iloc[-1],
                         "BB_Lower": historical_data['BB_Lower'].iloc[-1],
                         'BB_%B': historical_data['BB_%B'].iloc[-1],
                         'BB_BWidth': historical_data['BB_BWidth'].iloc[-1],
                         "closingPrice": historical_data["Close"].iloc[-1]   
                         })
            stock_data.append(data)

        except Exception as e:
            print(f"Error fetching data for {stocks}: {e}")

    # Create a DataFrame from the stock data
    df = pd.DataFrame(stock_data)
    df["MACDAS-dif"] = df["MACDAS"] - df["MACDAS_Signal"] 
    df["change"] = ((df["currentPrice"] / df["open"]) - 1) * 100
    df["BB_Pot"] = ((df['BB_Upper'] / df["currentPrice"]) - 1) * 100
    df["BB_Opt"] = ((df['BB_Lower'] / df["currentPrice"]) - 1) * 100
    df["TrendWay"] = np.select([(df["ADX"] > 20) & (df["ROC"] > 0), (df["ADX"] > 20) & (df["ROC"] <= 0), (df["ADX"] <= 20)], ["upper", "lower", "no-trend"], default="unknown")

    # Removed Columns "BB_Signal",'BB_Middle','BB_Upper','BB_Lower', "Senkou_Span_A","IchiSignal", "Senkou_Span_B", "Tenkan_sen", "Kijun_sen","OBV", "CMF", "AD", "TrendWay","priceToBook","BB_Upper","BB_Middle", "BB_Lower" "BB_Pot","BB_Opt",
    #df1 = df[["symbol","MACDAS-dif","MACDAS","MACDAS_Signal", 'BB_BWidth', 'BB_%B', 'fiftyTwoWeekLow', "closingPrice", 'fiftyTwoWeekHigh', "RSI", "ADX", "CCI", "ROC", "ATR"]]
    #removed for backtesting ["change",]
    df2 = df.rename(columns={"fiftyTwoWeekHigh": "YHigh", "fiftyTwoWeekLow": "YLow"})
    df2["date"] = endDate
    pd.set_option('display.float_format', '{:.2f}'.format)
    #print("\nDownload process is done!")
    df2
    now = datetime.now()  # This part will copy for our sell lists. Especially further analysis.
    formatted_time = now.strftime("%m-%d_%H-%M-%S")
    df2.to_csv(f"CSVs/Backtesting/{stocks}_{endDate}.csv")
    df2


In [10]:
bist1 = ["BIMAS.IS"]
endDate   = "2025-01-01"
startDate = "2024-05-01"
date_based_test(startDate,endDate,bist1)

In [6]:
#// ... existing code ...
bist1 = ["BIMAS.IS"]
endDate   = "2024-08-04"
startDate = "2024-05-01"

# Data downloading and processing cell
def date_based_test(startDate,endDate,bist1):
    stock_dict = {}
    stock_data = []
    stock_number = 0
    for stocks in bist1:
        try:
            ticker = yf.Ticker(stocks)
            stock_info = ticker.info
            data = {key: stock_info.get(key, None) for key in ["symbol", "dayLow", "dayHigh", "recommendationKey", 'fiftyTwoWeekLow', 'fiftyTwoWeekHigh']}
            #historical_data = ticker.history(period="1y") #"priceToBook", "currentPrice", "targetHighPrice", "targetLowPrice", "targetMeanPrice", "targetMedianPrice","bookValue", "open", 
            historical_data = ticker.history(start=startDate, end=endDate)  # Get historical data for the stock
            stock_dict[stocks] = historical_data
            if historical_data.empty:
                print(f"No historical data for {stocks}, skipping...")
                continue

            stock_number = 1 + stock_number
            #print(f"\r{stock_number}/{len(bist)} Downloaded {stocks} data", end='', flush=True)

            # Create a strategy with verified indicators
            MyStrategy = ta.Strategy(
                name="Core Indicators",
                description="Strategy using core technical indicators",
                ta=[
                    # Momentum Indicators
                    {"kind": "rsi"},
                    {"kind": "macd"},
                    {"kind": "cci"},
                    {"kind": "roc"},
                    {"kind": "stoch"},
                    {"kind": "willr"},
                    {"kind": "mom"},

                    # Trend Indicators
                    {"kind": "adx"},
                    {"kind": "ema", "length": 20},
                    {"kind": "sma", "length": 20},
                    {"kind": "tema"},

                    # Volatility Indicators
                    {"kind": "bbands"},
                    {"kind": "atr"},
                    {"kind": "natr"},
                    {"kind": "kc"},

                    # Volume Indicators
                    {"kind": "obv"},
                    {"kind": "cmf"},
                    {"kind": "mfi"},
                    {"kind": "vwap"},
                    {"kind": "ad"},

                    # Oscillator Indicators
                    {"kind": "ppo"},
                    {"kind": "stochrsi"}
                ]
            )

            # Calculate all indicators
            historical_data.ta.strategy(MyStrategy)

            # Get the most recent values for all indicators
            latest_data = historical_data.iloc[-1].to_dict()

            # Update data dictionary with all indicator values
            data.update({
                key: latest_data[key] 
                for key in latest_data.keys() 
                if key not in ['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
            })

            stock_data.append(data)

        except Exception as e:
            print(f"\nError fetching data for {stocks}: {e}")

    # Create DataFrame and add derived calculations
    df = pd.DataFrame(stock_data)

    # Add custom calculations
    #df["change"] = ((df["currentPrice"] / df["open"]) - 1) * 100
    # Save results
    now = datetime.now()
    formatted_time = now.strftime("%m-%d_%H-%M-%S")
    #df.to_csv(f"CSVs/Backtesting/{bist1[0]} {formatted_time}.csv")
    with open(f'CSVs/Backtesting/Total.csv', 'a', newline='') as f:
        df.to_csv(f, header=False, index=False)

    # Display results
    pd.set_option('display.float_format', '{:.2f}'.format)
    #print("\nDownload process is done!")
    df

#// ... existing code ...

In [ ]:
# we have to develop a dataframe for this

bist1 = ["TTRAK.IS"]
#endDate   = "2024-11-24"
endDate   = "2025-03-30"
spanDate  = "2025-02-01" 
startDate = "2024-09-01" #It must be 2 months before the span date

def generate_date_range(start_date: str, end_date: str) -> list:
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    
    date_list = [(start + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((end - start).days + 1)]
    return date_list
endDateList = generate_date_range(spanDate, endDate)

for endDate in endDateList:
    time.sleep(0.3)
    date_based_test(startDate= startDate, endDate= endDate,bist1=bist1)
    print(f"{endDate} was downloaded.")
print("Process is done!")

In [ ]:
#Gather all CSV files in the current directory
csv_files = glob.glob("CSVs/Backtesting/*.csv")
now = datetime.now()
now = now.strftime("%m-%d_%H-%M-%S")
#Read and combine all files into a single DataFrame
df_combined = pd.concat(
(pd.read_csv(f) for f in csv_files),
ignore_index=True
)
#Export the combined data to a new CSV file
#df_combined.to_csv(f"CSVs/Combine/{now}combined.csv", index=False)
df_combined.to_csv(f"CSVs/Combine/{bist1[0]}_combined_{now}.csv", index=False)
print("Process is done!")
